In [46]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import random
import re
import pandas as pd

In [3]:
url = "https://pdt.iuh.edu.vn/tra-cuu-hoc-bong/"
driver = webdriver.Firefox()
driver.get(url)

In [55]:
with open(r'C:\Users\HPs\Desktop\DSLOP\k19\DSTXT\KHMT19tt.txt', 'r', encoding='utf-8') as file:
    # Read the contents of the file
    contents = file.read()
pattern = r'(\d+)\s+(\d+)\s+([\w\s]+)\s+(Nam|Nữ)\s+(\d{2}/\d{2}/\d{4})\s+(\w+)'

# Find all matches in the large string
matches = re.findall(pattern, contents)

student = []
for match in matches:
    temp = {}
    stt, student_id, name, gender, dob, class1 = match
    temp["STT"] =  stt.replace("('", "")
    temp["ID"]  = student_id.replace("('", "")
    temp["Name"]= name.replace("('", "")
    temp["Gender"]= gender.replace("('", "")
    temp["DOB"] =  str(dob)
    temp["Class1"] =  class1
    student.append(temp)
ds_lop = pd.DataFrame(student)
ds_lop

,STT,ID,Name,Gender,DOB,Class1
0,1,23638721,Phan Hoài Bảo,Nam,21/10/2005,DHTH19ATT
1,2,23722781,Trương Tuấn Kiệt,Nam,23/04/2005,DHTH19ATT
2,3,23735431,Đặng Nhật Minh,Nam,20/01/2005,DHTH19ATT
3,4,23633591,Nguyễn Trường Tân,Nam,03/10/2005,DHTH19ATT
4,5,23635461,Kiều Đỗ Thuỷ Tiên,Nữ,28/06/2005,DHTH19ATT
...,...,...,...,...,...,...
116,117,23692321,Phạm Quang Khải,Nam,11/11/2005,DHTH19FTT
117,118,23634251,Nguyễn Tuấn Đạt,Nam,09/11/2005,DHTH19ATT
118,119,23733161,Nguyễn Trí Thức,Nam,02/07/2005,DHTH19GTT
119,120,23699451,Trần Đức Nguyên,Nam,10/06/2005,DHTH19GTT


In [56]:
ds = []
for i in ds_lop["ID"]:
    temp = {}
    mssv = driver.find_element(By.XPATH,'//*[@id="maSV"]')
    mssv.send_keys(i)
    driver.find_element(By.XPATH,'//*[@id="timkiem_btn"]').click()
    time.sleep(random.uniform(0.2, 0.4))
    try:
        score   = driver.find_element(By.XPATH,"/html/body/div[4]/div[1]/div[1]/div[1]/div[1]/div[1]/div[4]/article/div/div[3]/div/div[1]/div[4]")
        percent = driver.find_element(By.XPATH,"/html/body/div[4]/div[1]/div[1]/div[1]/div[1]/div[1]/div[4]/article/div/div[3]/div/div[2]/div[1]")
        temp = {"ID": i, "Score": score.text, "Percent": percent.text}
    except:
        temp = {"ID": i, "Score": "0", "Percent": "0"}
    temp["Class"] = "KHMT19TT"
    ds.append(temp)
    mssv.clear()    
DS_BACK_UP = ds
ds = pd.DataFrame(ds)  
ds["Score"] = ds["Score"].str.replace("Điểm: ","")
ds["Percent"] = ds["Percent"].str.replace("Mức học bổng: ","")  
final = pd.merge(ds_lop, ds, on="ID", how="left")
final 

,STT,ID,Name,Gender,DOB,Class1,Score,Percent,Class
0,1,23638721,Phan Hoài Bảo,Nam,21/10/2005,DHTH19ATT,0,0,KHMT19TT
1,2,23722781,Trương Tuấn Kiệt,Nam,23/04/2005,DHTH19ATT,0,0,KHMT19TT
2,3,23735431,Đặng Nhật Minh,Nam,20/01/2005,DHTH19ATT,0,0,KHMT19TT
3,4,23633591,Nguyễn Trường Tân,Nam,03/10/2005,DHTH19ATT,0,0,KHMT19TT
4,5,23635461,Kiều Đỗ Thuỷ Tiên,Nữ,28/06/2005,DHTH19ATT,3.45,30%,KHMT19TT
...,...,...,...,...,...,...,...,...,...
116,117,23692321,Phạm Quang Khải,Nam,11/11/2005,DHTH19FTT,0,0,KHMT19TT
117,118,23634251,Nguyễn Tuấn Đạt,Nam,09/11/2005,DHTH19ATT,0,0,KHMT19TT
118,119,23733161,Nguyễn Trí Thức,Nam,02/07/2005,DHTH19GTT,0,0,KHMT19TT
119,120,23699451,Trần Đức Nguyên,Nam,10/06/2005,DHTH19GTT,0,0,KHMT19TT


In [57]:
Link = rf'C:\Users\HPs\Desktop\DSLOP\crawl_dslop\crawl{final["Class"][0]}19.xlsx'
final.to_excel(Link, index = False)